In [14]:
# https://www.ilsole24ore.com/
# https://www.ilmeteo.it/portale/archivio-meteo/Milano/2023/Maggio/15
# https://www.3bmeteo.com/meteo/milano/storico/202305
# https://pollution.it/it/download/?noredirect=it-IT

# https://www.ilmeteo.it/portale/archivio-meteo/Milano/2000/Febbraio/18


In [15]:
%%capture
%pip install numpy
%pip install pandas
%pip install requests
%pip install matplotlib
%pip install BeautifulSoup
%pip install pyplot
%pip install openpyxl

In [16]:
import plotly.express as px
import plotly.graph_objects as go
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import json
from datetime import date
import threading
import time
import locale
import unicodedata


In [17]:
#imposto il formato dei dati di tipo "date"
locale.setlocale(locale.LC_TIME, 'it_IT')

'it_IT'

In [18]:
città = "Vigo+di+Cadore"
url = f"https://www.ilmeteo.it/portale/archivio-meteo/{città}/"
errori = []
mesi = {
    "gennaio": 31,
    "febbraio": 28,  #da ricordarsi il mese bisestile
    "marzo": 31,
    "aprile": 30,
    "maggio": 31,
    "giugno": 30,
    "luglio": 31,
    "agosto": 31,
    "settembre": 30,
    "ottobre": 31,
    "novembre": 30,
    "dicembre": 31
}
anni = []
for i in range(2000,2023+1):
    anni.append(str(i))

In [19]:
def extract_table(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    try:
        tables = soup.find_all('table') 
        
        #seleziono la tabella con i dati d'interesse
        table = tables[3]
        
        #estrapolo le info dalle righe della tavella
        rows = table.find_all('tr')
        
        # estraggo le intestazioni delle colonne
        headers = [header.text.strip() for header in rows[0].find_all('th')]
        
        # estraggo i dati delle celle
        data = []

        #salto il primo dato che sono gli headers
        for row in rows[1:]:
            #prendo il contenuto della singola cella <td> nella riga <tr>
            row_data = [cell.text.strip() for cell in row.find_all('td')]
            if len(row_data) == len(headers):
                data.append(row_data)
        # creo il DF utilizzando le intestazioni delle colonne e i dati delle celle
        df = pd.DataFrame(data, columns=headers)
        return(df)
    except IndexError:
        errori.append(f"L'url {url} non ha dati per quella giornata")
        return False

In [20]:
def save_json(dataframe):
    #estrarre la data dal dataframe
    date = dataframe.columns[0]
    date = date.replace(" ", "_")

    #creo la cartella se non esiste
    folder_path = "./meteo"
    os.makedirs(folder_path, exist_ok=True)
    
    #salvo il file prendendo la data come nome
    file_name = f"{date}.json"
    file_path = os.path.join(folder_path, file_name)
    
    #converto il dataframe in dizionario
    data_dict = dataframe.to_dict()
    
    # Salvataggio del dizionario come file JSON
    with open(file_path, "w") as json_file:
        json.dump(data_dict, json_file, ensure_ascii=False)
    
    print(f"Il dataframe è stato salvato come file JSON: {file_path}")



In [21]:

def get_days_to_download():
    oggi = date.today()
    days_to_download = []

    #costruiamo il link che scorre anni/mesi/giorni
    for anno in anni:
        url_a = url + anno + "/"
        if int(anno) % 4 == 0:
            mesi["febbraio"] = 29
        else:
            mesi["febbraio"] = 28
        for n_mese, mese in enumerate(mesi, start=1):
            url_m = url_a + str(mese) + "/"
            for giorno in range(1, mesi[mese] + 1):
                if date(int(anno), n_mese, giorno) <= oggi:
                    file_name = f"{giorno}_{mese}_{anno}.json"
                    file_path = os.path.join("./meteo", file_name)
                    if not os.path.isfile(file_path):
                        url_g = url_m + str(giorno) + "/"
                        days_to_download.append(url_g)

    return days_to_download




In [22]:
def download_data(url):
    try:
        df = extract_table(url)
        save_json(df)
    except:
        if df is not(False):
            print("Errore")
            errori.append(f"L'url {url} non è stato salvato.")


# Ottieni la lista dei giorni da scaricare
days_to_download = get_days_to_download()


In [23]:
# Scarica i dati in file JSON utilizzando multithreading
try:
    threads = []
    print(f"Sono da scaricare n{len(days_to_download)} file, inizierà ora il download")
    time.sleep(3)
    for i, link in enumerate(days_to_download):
        thread = threading.Thread(target=download_data, args=(link,), name=f"Thread-{i}")
        threads.append(thread)

    for thread in threads:
        thread.start()
        print("starting thread  ", thread.name)
        
    for thread in threads:
        thread.join()
except:
    for link in days_to_download:
        print("Multithread disabilitato")
        download_data(link)

# Scrivi gli errori riscontrati durante il download
with open('./errori.txt', 'w') as file:
    for errore in errori:
        file.write(errore + '\n')


Sono da scaricare n311 file, inizierà ora il download
starting thread   Thread-0
starting thread   Thread-1
starting thread   Thread-2
starting thread   Thread-3
starting thread   Thread-4
starting thread   Thread-5
starting thread   Thread-6
starting thread   Thread-7
starting thread   Thread-8
starting thread   Thread-9
starting thread   Thread-10
starting thread   Thread-11
starting thread   Thread-12
starting thread   Thread-13
starting thread   Thread-14
starting thread   Thread-15
starting thread   Thread-16
starting thread   Thread-17
starting thread   Thread-18
starting thread   Thread-19
starting thread   Thread-20
starting thread   Thread-21
starting thread   Thread-22
starting thread   Thread-23
starting thread   Thread-24
starting thread   Thread-25
starting thread   Thread-26
starting thread   Thread-27
starting thread   Thread-28
starting thread   Thread-29
starting thread   Thread-30
starting thread   Thread-31
starting thread   Thread-32
starting thread   Thread-33
star

In [24]:
folder_path = "./meteo"
files = os.listdir(folder_path)
monthly_data = []

for file in files:
    if file.endswith(".json"):
        file_path = os.path.join(folder_path, file)
        day,month, year = file.split("_")[0],file.split("_")[1], file.split("_")[2].split(".")[0]
        
        with open(file_path, "r", encoding="Windows-1252") as f:
            json_data = json.load(f)
        
        month_data = json_data.get("Dati rilevati")
        month_index = json_data.get(f"{day} {month} {year}")
        
        combined_dict = {}

        for key in month_index:
            if key in month_data:
                value = month_data[key]
                value = unicodedata.normalize('NFKD', value).encode('ASCII', 'ignore').decode('utf-8')
                if "°" in value or "%" in value or "km" in value or "mb" in value:
                    unit_index = value.find(" ")
                    if unit_index != -1:
                        unit = value[unit_index + 1:]
                        value = value[:unit_index]
                        combined_dict[month_index[key] + " (" + unit + ")"] = value
                else:
                    combined_dict[month_index[key]] = value

        combined_dict["day"] = f"{day} {month} {year}"
        monthly_data.append(combined_dict)

dataframe = pd.DataFrame(monthly_data)
# Conversione della colonna "day" in formato data
dataframe['day'] = pd.to_datetime(dataframe['day'], format='%d %B %Y', errors='coerce')

# Ordinamento del DataFrame in ordine cronologico
dataframe = dataframe.sort_values('day')
print(dataframe.shape)

(8233, 34)


In [28]:
dataframe.head(5000)

,Temperatura media,Temperatura minima,Temperatura massima,Punto di rugiada,UmiditÃ media (%),VisibilitÃ media (km),VelocitÃ del vento media (km/h),VelocitÃ massima del vento (km/h),Raffica,Pressione media sul livello del mare,...,Velocità del vento media (km/h),Velocità massima del vento (km/h),Raffica (km/h),Visibilità media,UmiditÃ minima (%),UmiditÃ massima (%),UmiditÃ media,Umidità media,VisibilitÃ media,year
2790,-10.3 AC,-15 AC,-5 AC,-11.3 AC,92,10.8,NaN,NaN,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
5742,-6.3 AC,-11 AC,-1.4 AC,-7.8 AC,89,11.1,NaN,NaN,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
6417,-6.2 AC,-9.7 AC,-1.8 AC,-8.2 AC,86,10.8,NaN,NaN,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
6691,-6.9 AC,-10.2 AC,-1 AC,-9.7 AC,80,10.5,NaN,NaN,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
6961,-3.3 AC,-8.2 AC,2 AC,-6.5 AC,78,10.9,NaN,NaN,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5073,-8 C,-11 C,-4 C,0 C,NaN,NaN,NaN,NaN,-,NaN,...,6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013
5342,-6 C,-11 C,-1 C,0 C,NaN,NaN,NaN,NaN,-,NaN,...,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013
5594,-4 C,-8 C,1 C,0 C,NaN,NaN,NaN,NaN,-,NaN,...,6,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013
6114,-1 C,-2 C,0 C,0 C,NaN,NaN,NaN,NaN,-,NaN,...,11,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013


# <span style='background :white'>ANDAMENTO TEMP MEDIA MILANO 2000->2023 </span>


In [26]:
# Estrarre l'anno dal campo 'day'
dataframe['year'] = pd.DatetimeIndex(dataframe['day']).year

# Raggruppa i dati per anno e giorno e calcola la media delle temperature
daily_avg_temperature = dataframe.groupby(['year', pd.Grouper(key='day', freq='D')])['Temperatura media (°C)'].mean().reset_index()

fig = go.Figure()

# Aggiungi una traccia separata per ogni anno
for year in daily_avg_temperature['year'].unique():
    data_year = daily_avg_temperature[daily_avg_temperature['year'] == year]
    fig.add_trace(go.Scatter(x=data_year['day'], y=data_year['Temperatura media (°C)'],
                             name=str(year)))

# Layout del grafico con i nomi delle colonne e assi
fig.update_layout(
    title='Andamento annuale della temperatura media',
    xaxis_title='Anni',
    yaxis_title='Temperatura media (°C)'
)

# Il grafico
fig.show()

KeyError: 'Column not found: Temperatura media (°C)'

# <span style='background :white'>ANDAMENTO TEMP MAX MILANO 2000->2023 </span>


In [ ]:
# Estrarre l'anno dal campo 'day'
dataframe['year'] = pd.DatetimeIndex(dataframe['day']).year

# Calcolare la temperatura massima giornaliera media per anno
daily_max_temperature = dataframe.groupby(['year', 'day'])['Temperatura massima (°C)'].mean().reset_index()

fig = px.line(daily_max_temperature, x='day', y='Temperatura massima (°C)', color='year')

# Layout del grafico con i nomi delle colonne e assi
fig.update_layout(
    title='Andamento annuale della temperatura massima',
    xaxis_title='Anni',
    yaxis_title='Temperatura massima (°C)'
)

# Il grafico
fig.show()

# <span style='background :white'>ANDAMENTO RAFFICHE VENTO MILANO 2000->2023 </span>


In [ ]:
# converto il valore in numero altrimenti python piange
dataframe['Velocità del vento media (km/h)'] = pd.to_numeric(dataframe['Velocità del vento media (km/h)'], errors='coerce')

# creo la colonna year
dataframe['Year'] = dataframe['day'].dt.year

# rimuovo i nan
dataframe_cleaned = dataframe.dropna(subset=['Velocità del vento media (km/h)'])

# dati riuniti e ripuliti
vento_medio_anno = dataframe_cleaned.groupby('Year')['Velocità del vento media (km/h)'].mean()

# grafico a barre tela
fig = go.Figure(data=[go.Bar(x=vento_medio_anno.index, y=vento_medio_anno.values)])

# calc la trend line                                                      polinomio di reg
coefficients = np.polyfit(vento_medio_anno.index, vento_medio_anno.values, 1)
trend_line = np.polyval(coefficients, vento_medio_anno.index)

#  trend line al grafico
fig.add_trace(go.Scatter(x=vento_medio_anno.index, y=trend_line, mode='lines', name='Trend line', line=dict(color='red', dash='dash')))

#etichette
fig.update_layout(xaxis_title='Anno', yaxis_title='Velocità media del vento (km/h)', title='Andamento della velocità media del vento per gli anni')

# grafico
fig.show()

# <span style='background :white'>ISTOGRAMMA DELLA VISIBILITÀ MEDIA PER ANNO A MI </span>


In [ ]:
# Filtra il dataframe per la colonna 'Visibilità media (km)' e rimuovi i valori nulli
dataframe_filtered = dataframe[dataframe['Visibilità media (km)'].notna()].copy()

# Rimuovi caratteri non numerici dalla colonna 'Visibilità media (km)'
dataframe_filtered.loc[:, 'Visibilità media (km)'] = dataframe_filtered['Visibilità media (km)'].str.replace('[^\d.]', '',regex=True)

# Converti la colonna 'Visibilità media (km)' in numerico
dataframe_filtered.loc[:, 'Visibilità media (km)'] = pd.to_numeric(dataframe_filtered['Visibilità media (km)'], errors='coerce')

# Calcola la visibilità media per anno
visibilita_per_anno = dataframe_filtered.groupby(dataframe_filtered['year'])['Visibilità media (km)'].mean()

# Crea il grafico a barre con Plotly
fig = go.Figure(data=[go.Bar(x=visibilita_per_anno.index, y=visibilita_per_anno.values)])

# Aggiungi etichette agli assi e al titolo del grafico
fig.update_layout(xaxis_title='Anno', yaxis_title='Visibilità media (km)', title='Istogramma della Visibilità Media per Anno a Milano TAAAC')

# Mostra il grafico
fig.show()

NEL 2010 DA MILANO SI POTEVA VEDERE IL K2 DA QUANTO ERA CLEAN LA VISIBILITA'

# <span style='background :white'>TEMP MAX LUGLIO MI YEAR BY YEAR 2000->2023 </span>


In [ ]:
# Converto la colonna 'Temperatura massima (°C)' in tipo numerico
dataframe['Temperatura massima (°C)'] = pd.to_numeric(dataframe['Temperatura massima (°C)'], errors='coerce')

# Creo la colonna 'Year' contenente il valore dell'anno
dataframe['Year'] = pd.to_datetime(dataframe['day']).dt.year

# Filtra i dati per il mese di luglio
dataframe_august = dataframe[dataframe['day'].dt.month == 7]

# Calcola la media della temperatura massima di luglio per ogni anno
media_temperatura_massima = dataframe_august.groupby('Year')['Temperatura massima (°C)'].mean()

# Crea l'istogramma
fig = go.Figure(data=[go.Bar(x=media_temperatura_massima.index, y=media_temperatura_massima.values)])

# Calcolo la linea di regressione
x = media_temperatura_massima.index
y = np.full_like(x, media_temperatura_massima.values.mean())
coefficients = np.polyfit(x, media_temperatura_massima.values, 1)
trendline = np.polyval(coefficients, x)

# Aggiungi la trend line al grafico come una linea rossa tratteggiata
fig.add_trace(go.Scatter(x=x, y=trendline, mode='lines', name='Trend line', line=dict(color='red', dash='dash')))

# Imposta le etichette degli assi e il titolo del grafico
fig.update_layout(xaxis_title='Anno', yaxis_title='Temperatura media massima di luglio (°C)', title='Andamento della temperatura massima di luglio per anno')

# Mostra il grafico
fig.show()


# <span style='background :white'>UMIDITA' % A MILANO 2000->2023 </span>


In [ ]:

# converto il valore in numero altrimenti Python piange
dataframe['Umidità media (%)'] = pd.to_numeric(dataframe['Umidità media (%)'], errors='coerce')

#  colonna 'Year'
dataframe['Year'] = dataframe['day'].dt.year

# rimuovo i valori NaN
dataframe_cleaned = dataframe.dropna(subset=['Umidità media (%)'])

#'umidità media per anno
umidità_medio_anno = dataframe_cleaned.groupby(dataframe_cleaned['Year'].astype(str))['Umidità media (%)'].mean()

# grafico a barre
fig = go.Figure(data=[go.Bar(x=umidità_medio_anno.index, y=umidità_medio_anno.values)])

# trend line
coefficients = np.polyfit(umidità_medio_anno.index.astype(int), umidità_medio_anno.values, 1)
trend_line = np.polyval(coefficients, umidità_medio_anno.index.astype(int))

# aggiungo la trend line al grafico
fig.add_trace(go.Scatter(x=umidità_medio_anno.index, y=trend_line, mode='lines', name='Trend line', line=dict(color='red', dash='dash')))

# etichette del grafico
fig.update_layout(xaxis_title='Anno', yaxis_title='Umidità percentuale', title='Umidità % negli anni')

# il grafico
fig.show()